# Step 0. Parameter Setting 

In [ ]:
import dtnmonitor_v2 as dtn
import os


### P1. SELECT THE MONITOR NETWORK INTERFACE
## used in Step 1,2,3 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
#dtn.interface = 'eth0.2038' 
dtn.interface = 'ens4.2038'
#dtn.interface = 'eth1'
#dtn.interface = 'all'


### P2. SELECT THE INPUT FILE , 
## used in Step 1 : file IO read file
file_read_fio = "/data/disk0/10G.dat"


### P3. NUTTCP SERVER IP 
## used in Step 2,3 : input nuttcp server's ip
nuttcp_server="10.250.38.52"

### P4. NUTTCP THREADS
## used in Step 2,3 :input threads number
count = 8

## run fio and check status
### select graph_mode [0|1] 
#graph_mode=0  # all, 
#graph_mode=1  # 100pt
graph_mode=0  

# Step 1. Test file IO

In [ ]:
# setup fio parameters
# first, filename is assigned with existed and >1GB file path 
randrepeat=" --randrepeat=0" #--randrepeat=0 
bs=" --bs=1M" #--bs=1M 
ioengine=" --ioengine=sync "#--ioengine=sync 
tuntime=" --runtime=30" #--runtime=30 
iodepth=" --iodepth=32"#--iodepth=32 
name=" --name=drive0"#--name=drive0 
size=" --size=100G "#--size=100G 
filename=" --filename="+file_read_fio #--filename=filename




In [ ]:

for i in range(count):
    filename=" --filename="+file_read_fio #--filename=filename
    cmd="fio --thread --rw=read --readonly --norandommap --group_reporting --time_based "
    command = cmd + randrepeat + bs + ioengine + tuntime + iodepth + name + size + filename +" & "
    print(command)
    os.system(command)

dtn.exec_command("sleep 30" ,graph_mode)

# Step 2. Test network transfer by memory

In [ ]:
command = ""

for i in range(count):
     command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
#    if i != (count-1):
#        command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
#    else: 
#        command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
        
#print(command)
os.system(command)
dtn.exec_command("sleep 40",graph_mode)



# Step 3. Test network transfer by disk

In [ ]:


for i in range(count):
    command = "nuttcp -t -i1 -s -xc "+str(i)+" -w 110M -P 5300"+str(i)+" -p 5310"+str(i)+ " "+ nuttcp_server +" < "+ file_read_fio +" & "
    #print(command)
    os.system(command)
    #dtn.exec_sysout(command)


dtn.exec_command("sleep 40",graph_mode)